In [2]:
import torch
import cv2
import tqdm
import matplotlib as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
meta_file = '../data/HAM10k/HAM10000_metadata.csv'
meta = pd.read_csv(meta_file)
meta.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [41]:
X, y = meta.iloc[:,[0,1,3,4,5,6]], meta.iloc[:,2]

In [42]:
root_path = '../data/HAM10k/HAM10000_images/'

# train/test Split
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.20, stratify=y)
print(f"train: {len(xtrain)}, test: {len(xtest)} ")

# Concat X/y for train/test
xtrain.insert(loc = 2, column = 'dx', value = ytrain)
xtest.insert(loc= 2 , column = 'dx', value = ytest )

# Adding Image paths column
xtrain['image_pth'] = root_path + xtrain['image_id'] + str('.jpg')
xtest['image_pth'] = root_path + xtest['image_id'] + str('.jpg')

# Writing to CSV
xtest.to_csv('../data/test.csv', index=False)
xtrain.to_csv('../data/train.csv', index=False)

train: 8012, test: 2003 


In [16]:
df = meta
desc = meta['dx'].value_counts()
desc


nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: dx, dtype: int64

In [18]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from tqdm import tqdm_notebook as tqdm

gen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, 
                        shear_range=0.15, zoom_range= 0.1, channel_shift_range= 10, horizontal_flip= True)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from tqdm import tqdm_notebook as tqdm

gen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, 
                        shear_range=0.15, zoom_range= 0.1, channel_shift_range= 10, horizontal_flip= True)

for key in desc.keys():
    print(key, desc[key])
    ratio = int(desc['nv']/ desc[key])
    print('ratio to nv: ', ratio)
    print('We need around', ratio, ' augmentation per image to change ratio to 1')
    all_images = df[df['dx'] == key]['image_id'].values
    if(key == 'nv'):
    continue
    printonce= True
    # iterate over all images augment them, save them and insert them in our metadata frame
    for image_ in tqdm(all_images):
    if(len(df[df['dx'] == key]) > len(df[df['dx'] == 'nv'])):
        if printonce:
            print(key, 'datapoints = ', len(df[df['dx'] == key]), 'reached above nv skipping more augmentations..')
            printonce = False
        continue
    try:
        basedir= 'data/HAM10k/HAM10000_images/'
        image_path=  basedir + image_ + '.jpg'
        image= load_img(image_path)
    except:
        print(f"Exception for image folder....")

    image = np.expand_dims(img_to_array(image), axis= 0)
    generated = gen.flow(image)
    row = df[df['image_id'] == image_]
    dict_for_df = {
        'lesion_id':row.lesion_id.values[0], 'image_id':row.image_id.values[0], 
        'dx':row.dx.values[0], 'dx_type':row.dx_type.values[0] ,
        'age':row.age.values[0], 'sex':row.sex.values[0], 'localization':row.localization.values[0] 
    }
    for i in range(int(ratio)):
        aug_image= next(generated).astype(np.uint8)
        # save this image with an underscore
        # add this to metadata dataframe
        image_name= dict_for_df['image_id'] + '_' + str(i)
        dict_for_df['image_id'] = image_name
        df = df.append(dict_for_df, ignore_index=True)
        plt.imsave('../data/HAM10k/Augmented/' + image_name + '.jpg', aug_image[0])